Threads:

https://www.cs.bu.edu/fac/snyder/publications/UnifChapter.pdf chapter from handbook
huh. wayne snyder is in boston
[Jouannaud and Kirchner 1991, 
Baader and Siekmann 1994 https://dl.acm.org/doi/abs/10.5555/185705.185711

approximate unification - we can minsat the multiequations. Or consider Least squares solutions as an anlogy. Put some kind of metric on how badly an equation is viotlated rather that descirtee 0 1. maybe go neural. neural logic pgroamming. neural unification


Bottom up eunification - Guess over a database what you'll unify on.

E-unification and narrowing
HO pattern unify
Full HO unify

I've said that higher order != has syntactic lambdas. Higher order unification could maybe seen as E-unify modulo combinator equations?

Unification modulo egraphs


boolean unification

Rational tree unification. Non well founded equations. Rationals themselves as coalgebra?

most general unifier as a greatest fixed point? Greatest something.

Categorical perspectives on substitutions.
Arrows. sure. between what? Variable sets? Term sets? Something else?


Guassian elimination means two things.
Add rows. - This is more like knuth bendix. ForAll uvars and bottom up, forward chaining
isolate variable and subst. More like unify. Exists uvars and top down, back chaining


equation breakdown. We could keep the old equations in a saturation process.
sin(x) = y --> x = arcsin(y)



Unify with branching as clauses?


https://www.cs.cmu.edu/Groups/fox/people/fp/papers/ppcp93.pdf  Higher-Order Logic Programming as Constraint Logic Programming
flex flex pairs as constraints


e-unify via ATP
factoring rules do the analog of solving unification (?) in a prolog
```
cnf(axiom, plus(X,Y) = plus(Y,X)). % The E-ness
cnf(conjecture, ~(f(X) = g(Y))).
```

ATP already unifies E-unification with knuth bendix
facotring rules
```
t != s | u = v
----------------
   s[u] = s[v]

```


Traat chapter uses something akin to multiset knuth bendix for Ac-unify

higher order unification vs higher order atp vs lambda prolog


As awlays 


I was suggesting in co-egraphs

unification modulo derivatives?
That plotkin thing
integrals?
trig?




In [ ]:
def comm_unify(vs,t1, t2):
    # todo quee is not in DNF. An and of ors...
    # Hmm. Not sufficient.
    # ok so an or of and ands
    todo = [[(t1,t2)]]
    if is_add(t):
        if is_add(s):
            if len(t.children()) = len(s.children())
                # The chilrren choices are correlated here.
                # ok, so we do need to
                # probably here we should delay this unification as long as possible
                todo = [todo.copy() + [(t[i], s[perm[i]])  for i in range(n) ] for perm in permutations(n)]
                


# unification arranged as a given clause algorithm
def given_clause_unify():


# Ematch
As always, we should consider ematching first.
Here again we get a flavor of unification's complication anyway and having a global queue to look at is useful.
We can delay hard destructions until later
prolog freeze





## AC

AC unify is exact opposite of AC KB?
Multiset something


matrices of subsitutions. "multply" is subst composition, add is multiset combine?


## Boolean
That recent typechekcing paper flix https://dl.acm.org/doi/10.1145/3485487 Relational nullable types with Boolean unification
https://www.youtube.com/watch?v=BPW-92b7j7A&ab_channel=ACMSIGPLAN fast and efficient boolean unification for hindley milner

boolean unification the sotry so far https://www.cs.rice.edu/~javaplt/411/23-spring/NewReadings/Unfication%20Theory/Boolean-unification---The-story-so-far_1989_Journal-of-Symbolic-Computation.pdf


Reduce to f(x) = g(x) becomes f(x) + g(x) = 0.

loweneim and booles method



## Generic Narrowing
